Code to compare divergence control with pure PID vs PID + open-loop accelerations to reach a desired setpoint.

In [ ]:
###Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

Admittedly, here we don't do it in the most efficient way.
So we:
1. Load pid and f_acc traces one by one
2. Downsample them
3. Cut regions that belong to the PID/f_acc sequences
4. Calculate error between set-point and actual divergence
5. Store data in the summary dataframe

In [ ]:
##Loading of a pair of the traces
pid=pd.read_csv('/home/myedut/SwitchDivServ/PID_vs_Open loop Acceleration/Acceleration_data/20250228_pid_5.csv')
f_acc=pd.read_csv('/home/myedut/SwitchDivServ/PID_vs_Open loop Acceleration/Acceleration_data/20250228_ffacc_5.csv')

In [ ]:
##Downsample
pid['time'] = pd.to_timedelta(pid['time'], unit='s')
pid.set_index('time', inplace=True)

pid_2=pid.resample('65ms').mean()

f_acc['time'] = pd.to_timedelta(f_acc['time'], unit='s')
f_acc.set_index('time', inplace=True)
f_acc_2=f_acc.resample('65ms').mean()

In [ ]:
#Make Time for the plot with an example of div_sp vs. div
Time=np.arange(0,len(pid_2)*0.065,0.065)

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(Time[:250],f_acc_2['divergence'].iloc[10:260].values,color='green', linewidth=3, label='open-loop acceleration+PID control')
plt.plot(Time[:250],pid_2['divergence'].iloc[:250].values,color='deeppink', linewidth=3, alpha=0.7,label='only PID control')
plt.plot(Time[:250],f_acc_2['divergence_sp'].iloc[10:260].values)
plt.plot(Time[:250],pid_2['divergence_sp'].iloc[:250].values,color='black', linewidth=3, label='divergence set-point')

#plt.legend(fontsize=22)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Time (s)', fontsize=22)
plt.ylabel('Divergence', fontsize=22)
ax = plt.gca()
#plt.gca().invert_xaxis() # Invert x-axis from 5 to 1

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig('lapse.png', dpi=300)

In [ ]:
#prvious day
pid_limits=[(76,125),(146,195)]
f_acc_limits=[(70,113),(143,187)]

In [ ]:
#1
pid_limits=[(68,124),(150,195),(220,250)]
f_acc_limits=[(70,110),(140,182),(212,255)]

In [ ]:
#2
pid_limits=[(55,119),(165,260)]
f_acc_limits=[(64,109),(137,240)]

In [ ]:
#3
pid_limits=[(100,150),(170,220)]
f_acc_limits=[(150,194),(222,260)]

In [ ]:
#4
pid_limits=[(60,120),(10,190),(210,260)]
f_acc_limits=[(165,205),(240,280),(310,350)]

In [ ]:
#5
pid_limits=[(55,123),(146,195),(220,267)]
f_acc_limits=[(64,110),(137,180),(210,250)]

In [ ]:
#6
pid_limits=[(86,145),(174,218)]
f_acc_limits=[(134,180),(207,251)]

In [ ]:
error_pid=np.abs(pid_2['divergence_sp'].values-pid_2['divergence'].values)
error_f_acc=np.abs(f_acc_2['divergence_sp'].values-f_acc_2['divergence'].values)

In [ ]:
filtered_values_pid = []

# Iterate over the pid_limits
for start, end in pid_limits:
    # Append the sliced values from error_pid to the filtered_values list
    filtered_values_pid.extend(error_pid[start:end+1])

filtered_values_f_acc = []

# Iterate over the f_acc_limits
for start, end in f_acc_limits:
    # Append the sliced values from error_f_acc to the filtered_values list
    filtered_values_f_acc.extend(error_f_acc[start:end+1])

In [ ]:
new_acc_values = {
    'mean_error': np.mean(filtered_values_f_acc),
    'std_error': np.std(filtered_values_f_acc),
    'median_error': np.median(filtered_values_f_acc)
}

new_pid_values = {
    'mean_error': np.mean(filtered_values_pid),
    'std_error': np.std(filtered_values_pid),
    'median_error': np.median(filtered_values_pid)
}

# Append new values to acc_summary
acc_summary = acc_summary.append(new_acc_values, ignore_index=True)

# Append new values to pid_summary
pid_summary = pid_summary.append(new_pid_values, ignore_index=True)


In [ ]:
pid_summary.to_csv('pid_summary.csv', index=False)
acc_summary.to_csv('acc_summary.csv', index=False)

In [ ]:
import scipy.stats as stats

# Extract the 'mean_error' columns
pid_mean_error = pid_summary['mean_error']
acc_mean_error = acc_summary['mean_error']

# Perform the paired t-test
t_stat, p_value = stats.ttest_rel(pid_mean_error, acc_mean_error)

# Print the results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

In [ ]:
pid_summary = pd.read_csv('pid_summary.csv')

In [ ]:
acc_summary = pd.read_csv('acc_summary.csv')